In [ ]:
!pip install numpy matplotlib scipy sklearn hmmlearn simplejson eyed3 pydub
!pip install pyAudioAnalysis
!pip install plot_metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 217 kB 14.7 MB/s 
     |████████████████████████████████| 130 kB 59.6 MB/s 
     |████████████████████████████████| 246 kB 62.4 MB/s 
     |████████████████████████████████| 242 kB 64.0 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=400b2a55cb33236d6224113aeb3df0efc492619bcd07840d6b93b51d1e824f8d
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.3 MB 1.9 MB/s 
  Created wheel for pyAudioAnalysis: filename=pyAudioAnalysis-0.3.14-py3-none-any.whl size=41264387 sha256=e470b0ff87a5bbec2f5817fbad18dfa81dd0d09b06440f43008bd08ff1c0b17a
  Stored in directory: /root/.cache/pip/wheels/85/b

In [ ]:
!pip install boto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 10.0 MB/s 


In [ ]:
import fnmatch
import os
import zipfile

from pyAudioAnalysis import audioBasicIO 
from pyAudioAnalysis import audioSegmentation as aS
import scipy.io.wavfile as wavfile
import wave

import numpy as np
from numpy.lib import stride_tricks
import os
from PIL import Image
import scipy.io.wavfile as wav

import pandas as pd

import randomS
import boto

from sklearn.metrics import confusion_matrix, roc_curve, auc
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def remove_silence(filename, out_dir, smoothing=1.0, weight=0.3, plot=False):
    """
    A function that implements pyAudioAnalysis' silence extraction module
    and creates wav files of the participant specific portions of audio. The
    smoothing and weight parameters were tuned for the AVEC 2016 dataset.

    Parameters
    ----------
    filename : filepath
        path to the input wav file
    out_dir : filepath
        path to the desired directory (where a participant folder will
        be created containing a 'PXXX_no_silence.wav' file)
    smoothing : float
        tunable parameter to compensate for sparseness of recordings
    weight : float
        probability threshold for silence removal used in SVM
    plot : bool
        plots SVM probabilities of silence (used in tuning)

    Returns
    -------
    A folder for each participant containing a single wav file
    (named 'PXXX_no_silence.wav') with the vast majority of silence
    and virtual interviewer speech removed. Feature extraction is
    performed on these segmented wav files.
    """
    partic_id = 'P' + filename.split('/')[-1].split('_')[0]  # PXXX
    if is_segmentable(partic_id):
        # create participant directory for segmented wav files
        participant_dir = os.path.join(out_dir, partic_id)
        if not os.path.exists(participant_dir):
            os.makedirs(participant_dir)

        os.chdir(participant_dir)

        [Fs, x] = audioBasicIO.read_audio_file(filename)
        segments = aS.silence_removal(x, Fs, 0.020, 0.020,
                                     smooth_window=smoothing,
                                     weight=weight,
                                     plot=plot)

        for s in segments:
            seg_name = "{:s}_{:.2f}-{:.2f}.wav".format(partic_id, s[0], s[1])
            wavfile.write(seg_name, Fs, x[int(Fs * s[0]):int(Fs * s[1])])

        # concatenate segmented wave files within participant directory
        concatenate_segments(participant_dir, partic_id)


def is_segmentable(partic_id):
    """
    A function that returns True if the participant's interview clip is not
    in the manually identified set of troubled clips. The clips below were
    not segmentable do to excessive static, proximity to the virtual
    interviewer, volume levels, etc.
    """
    troubled = set(['P300', 'P305', 'P306', 'P308', 'P315', 'P316', 'P343',
                    'P354', 'P362', 'P375', 'P378', 'P381', 'P382', 'P385',
                    'P387', 'P388', 'P390', 'P392', 'P393', 'P395', 'P408',
                    'P413', 'P421', 'P438', 'P473', 'P476', 'P479', 'P490',
                    'P492'])
    return partic_id not in troubled


def concatenate_segments(participant_dir, partic_id, remove_segment=True):
    """
    A function that concatenates all the wave files in a participants
    directory in to single wav file (with silence and other speakers removed)
    and writes in to the participant's directory, then removes the individual
    segments (when remove_segment=True).
    """
    infiles = os.listdir(participant_dir)  # list of wav files in directory
    outfile = '{}_no_silence.wav'.format(partic_id)

    data = []
    for infile in infiles:
        w = wave.open(infile, 'rb')
        data.append([w.getparams(), w.readframes(w.getnframes())])
        w.close()
        if remove_segment:
            os.remove(infile)

    output = wave.open(outfile, 'wb')
    # details of the files must be the same (channel, frame rates, etc.)
    output.setparams(data[0][0])

    # write each segment to output
    for idx in range(len(data)):
        output.writeframes(data[idx][1])
    output.close()

In [ ]:
dir_name = "/content/drive/MyDrive/DAIC-WOZ/depression-detection/data/raw/audio"
out_dir = "/content/drive/MyDrive/DAIC-WOZ/depression-detection/cleaned_files"

for file in os.listdir(dir_name):
  if file.endswith('.wav'):
    filename = os.path.join(dir_name, file)
    remove_silence(filename, out_dir)

/usr/local/lib/python3.7/dist-packages/pyAudioAnalysis/audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 26364844 bytes, expected 210918444 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


Error: ignored

In [ ]:
def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))
    # zeros at beginning (thus center of 1st window should be for sample nr. 0)
    frame = np.floor((frameSize)/2.0)
    frame = frame.astype(np.int64)
    size = np.zeros(frame)
    samples = np.append((size), sig)
    # cols for windowing
    cols = int(np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1)
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))

    frames = stride_tricks.as_strided(samples, shape=(cols, frameSize),
                                      strides=(samples.strides[0]*hopSize,
                                      samples.strides[0])).copy()
    frames *= win

    return np.fft.rfft(frames)


def logscale_spec(spec, sr=44100, factor=20.):
    """
    Scale frequency axis logarithmically.
    """
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))
    scale = scale.astype(int)

    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            newspec[:, i] = np.sum(spec[:, scale[i]:], axis=1)
        else:
            newspec[:, i] = np.sum(spec[:, scale[i]:scale[i+1]], axis=1)

    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[scale[i]:])]
        else:
            freqs += [np.mean(allfreqs[scale[i]:scale[i+1]])]

    return newspec, freqs


def stft_matrix(audiopath, binsize=2**10, png_name='tmp.png',
                save_png=False, offset=0):
    """
    A function that converts a wav file into a spectrogram represented by a \
    matrix where rows represent frequency bins, columns represent time, and \
    the values of the matrix represent the decibel intensity. A matrix of \
    this form can be passed as input to the CNN after undergoing normalization.
    """
    samplerate, samples = wav.read(audiopath)
    s = stft(samples, binsize)

    sshow, freq = logscale_spec(s, factor=1, sr=samplerate)
    ims = 20.*np.log10(np.abs(sshow)/10e-6)  # amplitude to decibel
    timebins, freqbins = np.shape(ims)

    ims = np.transpose(ims)
    ims = np.flipud(ims)  

    if save_png:
        create_png(ims, png_name)

    return ims


def create_png(im_matrix, png_name):
    """
    Save grayscale png of spectrogram.
    """
    image = Image.fromarray(im_matrix)
    image = image.convert('L')  # convert to grayscale
    image.save(png_name)

In [ ]:
dir_name = '/content/drive/MyDrive/DAIC-WOZ/depression-detection/cleaned_files'

# walks through wav files in dir_name and creates pngs of the spectrograms.
# This is a visual representation of what is passed to the CNN before
# normalization, although a cropped matrix representation is actually
# passed.
for subdir, dirs, files in os.walk(dir_name):
    for file in files:
        if file.endswith('.wav'):
          wav_file = os.path.join(subdir, file)
          png_name = subdir + '/' + file[:-4] + '.png'
          print('Processing '+ file + '...')
          stft_matrix(wav_file, png_name=png_name, save_png=True)

Processing P301_no_silence.wav...
Processing P302_no_silence.wav...
Processing P303_no_silence.wav...
Processing P304_no_silence.wav...
Processing P307_no_silence.wav...
Processing P309_no_silence.wav...
Processing P310_no_silence.wav...
Processing P311_no_silence.wav...
Processing P312_no_silence.wav...
Processing P313_no_silence.wav...
Processing P314_no_silence.wav...
Processing P317_no_silence.wav...
Processing P318_no_silence.wav...
Processing P319_no_silence.wav...
Processing P320_no_silence.wav...
Processing P321_no_silence.wav...
Processing P322_no_silence.wav...
Processing P323_no_silence.wav...
Processing P324_no_silence.wav...
Processing P325_no_silence.wav...
Processing P326_no_silence.wav...
Processing P327_no_silence.wav...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in log10


Processing P328_no_silence.wav...
Processing P329_no_silence.wav...
Processing P330_no_silence.wav...
Processing P331_no_silence.wav...
Processing P332_no_silence.wav...
Processing P333_no_silence.wav...
Processing P334_no_silence.wav...
Processing P335_no_silence.wav...
Processing P336_no_silence.wav...
Processing P337_no_silence.wav...
Processing P338_no_silence.wav...
Processing P339_no_silence.wav...
Processing P340_no_silence.wav...
Processing P341_no_silence.wav...
Processing P344_no_silence.wav...
Processing P345_no_silence.wav...
Processing P346_no_silence.wav...
Processing P347_no_silence.wav...
Processing P348_no_silence.wav...
Processing P349_no_silence.wav...
Processing P350_no_silence.wav...
Processing P351_no_silence.wav...
Processing P352_no_silence.wav...
Processing P353_no_silence.wav...
Processing P355_no_silence.wav...
Processing P356_no_silence.wav...
Processing P357_no_silence.wav...
Processing P358_no_silence.wav...
Processing P359_no_silence.wav...
Processing P36

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/DAIC-WOZ/train_split_Depression_AVEC2017.csv')
df_test = pd.read_csv('/content/drive/MyDrive/DAIC-WOZ/test_split_Depression_AVEC2017.csv')
df_dev = pd.concat([df_train, df_test], axis=0)

In [ ]:
df_train.head()

,Participant_ID,PHQ8_Binary,PHQ8_Score,Gender,PHQ8_NoInterest,PHQ8_Depressed,PHQ8_Sleep,PHQ8_Tired,PHQ8_Appetite,PHQ8_Failure,PHQ8_Concentrating,PHQ8_Moving
0,303,0,0,0,0,0,0.0,0,0,0,0,0
1,304,0,6,0,0,1,1.0,2,2,0,0,0
2,305,0,7,1,0,1,1.0,2,2,1,0,0
3,310,0,4,1,1,1,0.0,0,0,1,1,0
4,312,0,2,1,0,0,1.0,1,0,0,0,0


In [ ]:
def build_class_dictionaries(dir_name):
    """
    Builds a dictionary of depressed participants and non-depressed
    participants with the participant id as the key and the matrix
    representation of the no_silence wav file as the value. These
    values of this dictionary are then randomly cropped and sampled
    from to create balanced class and speaker inputs to the CNN.
    Parameters
    ----------
    dir_name : filepath
        directory containing participant's folders (which contains the
        no_silence.wav)
    Returns
    -------
    depressed_dict : dictionary
        dictionary of depressed individuals with keys of participant id
        and values of with the matrix spectrogram representation
    normal_dict : dictionary
        dictionary of non-depressed individuals with keys of participant id
        and values of with the matrix spectrogram representation
    """
    print("dir name: "+dir_name)
    depressed_dict = dict()
    normal_dict = dict()
    for subdir, dirs, files in os.walk(dir_name):
        for file in files:
            if file.endswith('no_silence.wav'):
                partic_id = int(file.split('_')[0][1:])
                if in_dev_split(partic_id):
                    wav_file = os.path.join(subdir, file)
                    # matrix representation of spectrogram
                    mat = stft_matrix(wav_file)
                    depressed = get_depression_label(partic_id)  # 1 if True
                    if depressed:
                        depressed_dict[partic_id] = mat
                    elif not depressed:
                        normal_dict[partic_id] = mat
    return depressed_dict, normal_dict

def in_dev_split(partic_id):
    """
    Returns True if the participant is in the AVEC development split
    (aka participant's we have depression labels for)
    """
    return partic_id in set(df_dev['Participant_ID'].values)

def get_depression_label(partic_id):
    """
    Returns participant's PHQ8 Binary label. 1 representing depression;
    0 representing no depression.
    """
    return df_dev.loc[df_dev['Participant_ID'] ==
                      partic_id]['PHQ8_Binary'].item()

In [ ]:
dir_name = '/content/drive/MyDrive/DAIC-WOZ/depression-detection/cleaned_files'
depressed_dict, normal_dict = build_class_dictionaries(dir_name)

dir name: /content/drive/MyDrive/DAIC-WOZ/depression-detection/cleaned_files


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in log10


In [ ]:
len(normal_dict)

32

In [ ]:
len(depressed_dict)

21

In [ ]:
np.mean(depressed_dict[372][0])

126.2002477444343

In [ ]:
np.mean(depressed_dict[372], axis=1)

array([126.20024774, 129.27683055, 129.46544186, 129.29263542,
       129.44042534, 129.70193448, 129.64078754, 129.71279247,
       129.82265087, 129.96922948, 130.03517676, 130.11897245,
       130.19364415, 130.25025579, 130.45038444, 130.5762402 ,
       130.77021701, 130.91828251, 130.91725107, 130.92185182,
       131.0946188 , 131.22353238, 131.28893832, 131.40364758,
       131.58458   , 131.77832244, 131.85734164, 131.96099997,
       132.12584083, 132.20479329, 132.35929961, 132.53856904,
       132.46995828, 132.65987244, 132.80714281, 133.09822574,
       133.25971202, 133.4305809 , 133.56895751, 133.75875706,
       133.84669248, 134.0291568 , 134.08494195, 134.17575577,
       134.26774113, 134.33765902, 134.52618655, 134.65526591,
       134.74048204, 134.75825161, 134.89974868, 134.96588157,
       134.92934837, 134.90299458, 134.99140182, 135.08325737,
       135.03473433, 135.19230265, 135.18638341, 135.10271491,
       135.16403119, 135.16540995, 135.179846  , 135.16

In [ ]:
depressed_dict[319].shape

(513, 8935)

In [ ]:
normal_dict[304].shape

(513, 11675)

In [ ]:
data = pd.DataFrame(columns=['feature', 'label'])

In [ ]:
data

,feature,label


In [ ]:
i=0
for entry in depressed_dict:
  features = np.mean(depressed_dict[entry], axis=1)
  data.loc[i] = [features, 1]
  i=i+1

In [ ]:
for entry in normal_dict:
  features = np.mean(normal_dict[entry], axis=1)
  data.loc[i] = [features, 0]
  i=i+1

In [ ]:
data.head()

,feature,label
0,"[120.9369828347032, 124.10003769474918, 124.24...",1
1,"[123.46521743780323, 126.39837745042487, 126.5...",1
2,"[118.73505362591818, 121.93769079267553, 122.2...",1
3,"[122.90534364773356, 125.84956975948043, 126.0...",1
4,"[120.48609165792581, 123.58268568717973, 123.7...",1


In [ ]:
df_features = pd.DataFrame(data['feature'].values.tolist())

In [ ]:
df_labels = pd.DataFrame(data['label'])

In [ ]:
newdf = pd.concat([df_features,df_labels], axis=1)

In [ ]:
newdf.head()

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,512,label
0,120.936983,124.100038,124.245988,123.927154,124.112402,124.308507,124.304275,124.179082,124.193281,124.419315,...,165.200116,168.640128,170.593622,168.503220,164.653732,161.515533,165.354719,169.537315,168.527139,1
1,123.465217,126.398377,126.513568,126.232078,126.564662,126.600454,126.687338,126.665864,126.663428,126.812157,...,165.711708,165.634871,166.444124,168.213004,170.495510,171.860799,175.179427,179.145702,177.770790,1
2,118.735054,121.937691,122.222802,121.753684,121.936010,122.204681,122.211522,122.392191,122.276255,122.365782,...,156.929456,153.763250,153.143993,155.464798,160.277405,161.313385,165.482314,170.169580,169.119508,1
3,122.905344,125.849570,126.006568,125.820990,125.975764,126.174310,126.202263,126.295669,126.278243,126.449474,...,157.857579,155.774969,156.414269,159.099916,162.613608,164.627227,170.291814,174.686052,172.737359,1
4,120.486092,123.582686,123.705956,123.279740,123.590537,123.707338,123.759219,123.741586,123.746968,123.760098,...,166.022148,169.035596,173.087298,172.979531,169.463524,165.211606,167.089659,169.885832,168.463253,1


In [ ]:
newdf.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN
newdf.dropna(inplace=True)

In [ ]:
newdf.head() # row 31 and 40 gets deleted

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,512,label
0,120.936983,124.100038,124.245988,123.927154,124.112402,124.308507,124.304275,124.179082,124.193281,124.419315,...,165.200116,168.640128,170.593622,168.503220,164.653732,161.515533,165.354719,169.537315,168.527139,1
1,123.465217,126.398377,126.513568,126.232078,126.564662,126.600454,126.687338,126.665864,126.663428,126.812157,...,165.711708,165.634871,166.444124,168.213004,170.495510,171.860799,175.179427,179.145702,177.770790,1
2,118.735054,121.937691,122.222802,121.753684,121.936010,122.204681,122.211522,122.392191,122.276255,122.365782,...,156.929456,153.763250,153.143993,155.464798,160.277405,161.313385,165.482314,170.169580,169.119508,1
3,122.905344,125.849570,126.006568,125.820990,125.975764,126.174310,126.202263,126.295669,126.278243,126.449474,...,157.857579,155.774969,156.414269,159.099916,162.613608,164.627227,170.291814,174.686052,172.737359,1
4,120.486092,123.582686,123.705956,123.279740,123.590537,123.707338,123.759219,123.741586,123.746968,123.760098,...,166.022148,169.035596,173.087298,172.979531,169.463524,165.211606,167.089659,169.885832,168.463253,1


In [ ]:
combined_df = newdf

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
X = combined_df.drop(['label'], axis=1)
y = combined_df.label
xxx = StratifiedShuffleSplit(1, test_size=0.2, random_state=12)
for train_index, test_index in xxx.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_train.shape

(40, 513)

In [ ]:
y_train

array([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1])

In [ ]:
X_train[0]

array([124.71814159, 127.59192554, 127.77647776, 127.45668368,
       127.66526167, 127.91124397, 127.83337068, 127.8389179 ,
       127.81577528, 127.81802303, 127.86488821, 128.12147854,
       128.33523245, 128.35126304, 128.59578447, 128.62341768,
       128.69099339, 128.7515971 , 128.66686918, 128.70042129,
       128.74763954, 128.95120297, 129.07970797, 129.17991234,
       129.29472845, 129.50089602, 129.51999959, 129.42548953,
       129.52594621, 129.64807748, 129.79982357, 129.86423773,
       130.00803347, 130.29576319, 130.34301468, 130.51835539,
       130.57738956, 130.88869413, 130.9028991 , 131.04128175,
       131.15669328, 131.36987757, 131.48660641, 131.64094874,
       131.79444525, 131.97914939, 132.217252  , 132.41605319,
       132.60981054, 132.83093719, 133.00680851, 133.13725479,
       133.31915737, 133.48450897, 133.81305869, 133.87646224,
       134.02548893, 134.20014165, 134.40403165, 134.64563798,
       134.70863563, 134.89405171, 135.10628031, 135.23

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
X_train

array([[ 0.46254109,  0.45416765,  0.47552254, ...,  1.38184197,
         1.11945081,  0.90223682],
       [-0.02176311, -0.02448368, -0.02288505, ...,  0.07630955,
        -0.07233405, -0.1186564 ],
       [-0.30733375, -0.34787437, -0.31896168, ..., -0.02213093,
         0.24263731,  0.39669267],
       ...,
       [ 1.48765869,  0.94674689,  0.68795646, ...,  1.45066558,
         1.76561656,  1.94486354],
       [ 0.00274781,  0.00546518,  0.0068009 , ...,  0.47500487,
         0.24549366,  0.02787445],
       [-0.81958958, -0.79862584, -0.81174088, ..., -1.65641306,
        -1.88865716, -1.83032961]])

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical

In [ ]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim=513))#Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [ ]:
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [ ]:
classifier.fit(X_train,y_train, batch_size=10, epochs=20)

Epoch 1/20
4/4 [==============================] - 0s 4ms/step - loss: 0.6933 - accuracy: 0.5250
Epoch 2/20
4/4 [==============================] - 0s 4ms/step - loss: 0.6927 - accuracy: 0.6000
Epoch 3/20
4/4 [==============================] - 0s 3ms/step - loss: 0.6924 - accuracy: 0.6000
Epoch 4/20
4/4 [==============================] - 0s 4ms/step - loss: 0.6920 - accuracy: 0.6000
Epoch 5/20
4/4 [==============================] - 0s 3ms/step - loss: 0.6916 - accuracy: 0.6000
Epoch 6/20
4/4 [==============================] - 0s 3ms/step - loss: 0.6908 - accuracy: 0.6000
Epoch 7/20
4/4 [==============================] - 0s 3ms/step - loss: 0.6899 - accuracy: 0.6000
Epoch 8/20
4/4 [==============================] - 0s 4ms/step - loss: 0.6888 - accuracy: 0.6000
Epoch 9/20
4/4 [==============================] - 0s 4ms/step - loss: 0.6871 - accuracy: 0.6000
Epoch 10/20
4/4 [==============================] - 0s 4ms/step - loss: 0.6854 - accuracy: 0.6000
Epoch 11/20
4/4 [======================

In [ ]:
classifier.save("/content/drive/MyDrive/DAIC-WOZ/depression-detection/cleaned_files/audio_model.h5")

In [ ]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

2/2 [==============================] - 0s 6ms/step - loss: 0.6483 - accuracy: 0.6000


[0.6483028531074524, 0.6000000238418579]

In [ ]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

1/1 [==============================] - 0s 43ms/step


In [ ]:
X_test.shape

(11, 513)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[6 0]
 [5 0]]
